# Анализ приложения для чтения книг

**Цель исследования:** проанализировать популярность авторов и издательств и сформулировать ценностное предложение для нового продукта.

Для начала имортируем нужные библиотеки и настроим параметры доступа к базе данных.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': '...', # имя пользователя
             'pwd': '...', # пароль
             'host': '...',
             'port': ..., # порт подключения
             'db': '...'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Для запросов нам понядобятся две функции. Одна будет возвращать результат запроса, а вторая 5 верхних строк таблиц из базы данных.

In [2]:
def query_result(query):
    '''
    Принимает запрос, а возвращает результат этого запроса
    '''
    return pd.io.sql.read_sql(query, con=engine)

In [3]:
def head(tables):
    '''
    Принимает список таблиц, возвращает название и 5 верхних строк каждой из них
    '''
    for table in tables:
        query = 'SELECT * FROM {} LIMIT 5'.format(table)
        display(table)
        display(query_result(query))

In [4]:
tables = ['books', 'authors', 'publishers', 'reviews', 'ratings']

In [5]:
head(tables)

'books'

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


'authors'

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


'publishers'

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


'reviews'

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


'ratings'

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Теперь выполним необходимые по ТЗ запросы.

#### Запрос 1. Посчитайте, сколько книг вышло после 1 января 2000 года.

In [6]:
query_1 = '''
            SELECT COUNT(DISTINCT(book_id))
              FROM books
             WHERE publication_date >= '2000-01-01';
        '''

In [7]:
result_1 = query_result(query_1)

In [8]:
print('В нашем приложении доступна {} книга, вышедшая после 1 января 2000 года.'.format(result_1.iloc[0,0]))

В нашем приложении доступна 821 книга, вышедшая после 1 января 2000 года.


В приложении неплохой выбор книг.

#### Запрос 2. Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [9]:
query_2 = '''
          SELECT books.title AS title,
                 count(distinct(review_id)) AS cnt_reviews,
                 avg(rating) AS avg_rating
            FROM books
                 LEFT JOIN reviews
                 ON books.book_id = reviews.book_id
                 LEFT JOIN ratings
                 ON books.book_id = ratings.book_id
        GROUP BY books.book_id;
        '''

In [10]:
result_2 = query_result(query_2)

Результат запроса — таблица с названием книги, количеством обзоров и средней оценкой. Выведем её первые 5 строк.

In [11]:
result_2.head(5)

,title,cnt_reviews,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000


Эту таблицу можно проанализировать отдельно, построить графики и сделать выводы.

#### Запрос 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [12]:
query_3 = '''
            SELECT SBQ.publisher
              FROM (SELECT publishers.publisher AS publisher,
                           COUNT(DISTINCT(books.book_id)) AS cnt_books
                      FROM books
                           LEFT JOIN publishers
                           ON books.publisher_id = publishers.publisher_id
                     WHERE books.num_pages > 50
                  GROUP BY publishers.publisher
                  ORDER BY COUNT(DISTINCT(books.book_id)) DESC
                     LIMIT 1) AS SBQ;
        '''

In [13]:
result_3 = query_result(query_3).iloc[0,0]

In [14]:
print('Издательство {} выпустило наибольшее число книг толще 50 страниц.'.format(result_3))

Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц.


#### Запрос 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [15]:
query_4 = '''
            SELECT authors.author AS author
              FROM books
                   LEFT JOIN ratings
                   ON books.book_id = ratings.book_id
                   LEFT JOIN authors
                   ON books.author_id = authors.author_id
             WHERE books.author_id IN
                   (SELECT books.author_id
                      FROM books
                           LEFT JOIN ratings
                           ON books.book_id = ratings.book_id
                  GROUP BY books.book_id,
                           books.author_id
                    HAVING count(ratings.rating_id) >= 50)
          GROUP BY authors.author
          ORDER BY AVG(ratings.rating) DESC
             LIMIT 1;
        '''

In [16]:
result_4 = query_result(query_4).iloc[0,0]

In [17]:
print('{} имеет самую высокую среднюю оценку книг с 50 и более оценками.'.format(result_4))

J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку книг с 50 и более оценками.


Книги этого автора больше всего нравятся пользователям приложения.

#### Запрос 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [18]:
query_5 = '''
            SELECT AVG(SBQ.count) AS avg_count
              FROM (SELECT username,
                           COUNT(review_id) AS count
                      FROM reviews
                     WHERE username IN
                           (SELECT username
                              FROM ratings
                          GROUP BY username
                            HAVING COUNT(rating_id) > 50)
                  GROUP BY username) as SBQ;
        '''

In [19]:
result_5 = round(query_result(query_5).iloc[0,0])

In [20]:
print('Пользователи, которые поставили больше 50 оценок, в среднем написали {} обзора.'.format(result_5))

Пользователи, которые поставили больше 50 оценок, в среднем написали 24 обзора.
